# Простой скилл для чат-бота на DeepPavlov

В данном тюториале представлен один из самых простых вариантов чат-бота для разговора на общие тематики.
Меняя/дополняя датасет для модели ответов на часто задаваемые вопросы, пользователь может улучшить своего чат-бота.

Установим библиотеку и основные используемые зависимости. Также скачаем необходимые файлы.

In [ ]:
!pip install deeppavlov
!python -m deeppavlov install rusentiment_convers_bert
!python -m deeppavlov download rusentiment_convers_bert
!python -m deeppavlov download ner_rus_bert
!pip install pandas==1.0.1

## Анализ тональности

Первым делом поднимем модель для анализа тональности на русском языке. Возьмем BERT-based модель для классификации, обученную на датасете постов/комментариев Вконтакте. BERT модель предобучена на большом корпусе русских разговорных текстов (BERT RU conversational).
http://docs.deeppavlov.ai/en/master/features/models/bert.html

In [ ]:
from deeppavlov import build_model
from deeppavlov import configs


sentiment_classifier =  build_model(configs.classifiers.rusentiment_convers_bert)

Посмотрим, что предсказывает модель. Модели на вход надо подавать батч (batch - лист примеров) текстовых строк, которые мы хотим проанализировать на тональность.

In [8]:
sentiment_classifier(["Привет", "Дела отлично, собираюсь на работу.", "Классные часы.", "Даже не знаю.", "Все отстой!"])

['speech', 'positive', 'positive', 'neutral', 'negative']

## Именованные сущности

Также нам понадобится модель распознавания именованных сущностей для извлечения имен, названий городов и организаций.

In [ ]:
ner =  build_model(configs.ner.ner_rus_bert)

In [96]:
ner(["Кто такой Владимир Ленин?",  
     "Была в ПАО Сбербанке, открыла счет.", 
     "Видела такой в Вене в Австрии."])

[[['Кто', 'такой', 'Владимир', 'Ленин', '?'],
  ['Была', 'в', 'ПАО', 'Сбербанке', ',', 'открыла', 'счет', '.'],
  ['Видела', 'такой', 'в', 'Вене', 'в', 'Австрии', '.']],
 [['O', 'O', 'B-PER', 'I-PER', 'O'],
  ['O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O'],
  ['O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O']]]

## Часто задаваемые вопросы 

Frequently asked questions (FAQ) - модель ответов на вопросы, ответы к которым присутствуют в датасете.

In [65]:
from pandas import DataFrame

pairs = [["Привет.", "Привет!))"], 
         ["Добрый день!", "Привет!))"], 
         ["Здравствуйте!", "Привет!))"], 
         ["Доброе утро!", "Привет!))"], 
         ["Добрый вечер!", "Привет!))"], 
         ["Как дела?", "Замечательно! А у тебя как дела?"], 
         ["Как твои дела?", "Замечательно! А у тебя как дела?"],
         ]

# соберем датафрейм из имеющихся данных
df = DataFrame(pairs, columns=["Question", "Answer"])
# сохраним данные в файл, так как они понадобятся нам для обучения нашей модели
df.to_csv("/my_data.csv", index=False)

Возьмем имеющийся в библиотеке конфигурационный файл, содержащий в себе пайплайн для модели FAQ, основанной на cosine similarity. 

В файле надо заменить путь к датасету на тот, что мы определили выше, сохранив датасет. Также заменим путь к файлам моделей, чтобы не подгрузить/не перезаписать скачанные модели из данного конфига.

In [67]:
import json

faq_config_path = configs.faq.tfidf_logreg_autofaq

with open(faq_config_path, "r") as f:
  faq_config = json.load(f)

In [68]:
faq_config["dataset_reader"]

{'class_name': 'faq_reader',
 'data_url': 'http://files.deeppavlov.ai/faq/school/faq_school.csv',
 'x_col_name': 'Question',
 'y_col_name': 'Answer'}

In [69]:
faq_config["dataset_reader"]["data_url"] = None
faq_config["dataset_reader"]["data_path"] = "/my_data.csv"
faq_config["metadata"]["variables"]["MODELS_PATH"] = "{ROOT_PATH}/my_models"

Обучим модель FAQ на сохраненных данных.

In [ ]:
from deeppavlov import train_model

faq_model = train_model(faq_config)

Потестируем получившуюся модель.

In [113]:
faq_model(["привет", 
           "кто ты?", 
           "как ты?",
           "какие дела?", 
           "как твои дела?"])

[['Привет!))',
  'Привет!))',
  'Замечательно! А у тебя как дела?',
  'Замечательно! А у тебя как дела?',
  'Замечательно! А у тебя как дела?'],
 [[0.996982189162366, 0.0030178108376339515],
  [0.9430167449356117, 0.05698325506438824],
  [0.048573286561535145, 0.9514267134384649],
  [0.048573286561535145, 0.9514267134384649],
  [0.00463466085327624, 0.9953653391467238]]]

### Следует отметить, что данная модель FAQ не multi-label, то есть нет варианта "не принадлежит ни к одному классу", поэтому будем использовать пороговое значение для уверенности (например, 0.96).

In [117]:
def generate_response(text):
  faq_classification = faq_model([text])
  faq_probas = faq_classification[1][0]
  if any([p > 0.96 for p in faq_probas]):
    # если faq модель сильно уверена в ответе, выдаем ее ответ пользователю
    return faq_classification[0][0]

  # определим тональность пришедшего на вход сообщения
  sentim = sentiment_classifier([text])[0]
  # определим именованные сущности в сообщении
  entities = ner([text])
  if any([ent != "O" for ent in entities[1][0]]):
    # если хотя бы один из токенов помечен как именованная сущность
    # здесь мы делаем послабление, что сущность у нас одна,
    # и слепляем из нескольких токенов одну сущность
    entity = " ".join([ent for ent, ent_type in 
                       zip(entities[0][0], entities[1][0]) 
                       if ent_type != "O"])
    if sentim == "positive":
      # если у нас есть именованная сущность и позитивный сентимент,
      # скажем, что нам тоже нравится эта сущность
      return f"Мне тоже нравится {entity}!"
    elif sentim == "negative":
      # если у нас есть именованная сущность и негативный сентимент,
      # скажем, что нам тоже не нравится эта сущность
      return f"Мне тоже не нравится {entity}!"
    else:
      # если у нас есть именованная сущность,
      # спросим, что пользователь думает о ней
      return f"Что ты думаешь о {entity}?"
  if sentim == "positive":
    # если у нас нет именованных сущностей и есть позитивный сентимент,
    # скажем классно
    return "Классно!"
  elif sentim == "negative":
    # если у нас нет именованных сущностей и есть позитивный сентимент,
    # скажем, что нам жаль
    return "Очень жаль.."
  # во всех остальных случаях, попросим рассказать нам что-нибудь
  return "Расскажи мне что-нибудь интересное, пожалуйста!"


In [115]:
generate_response("Ленин - классный мужик!")

'Мне тоже нравится Ленин!'

In [116]:
from ipywidgets import widgets
from IPython.display import display

text = widgets.Text()
display(text)

def handle_submit(sender):
  print(f"Вы: {text.value}")
  print(f"Бот: {generate_response(text.value)}")

text.on_submit(handle_submit)

Text(value='')